# Neural Network Framework Tutorial

This guide demonstrates how to use the custom neural network framework for various machine learning tasks.

## Contents:
1. Loading and Preparing Data
2. Creating Neural Network Architecture
3. Training Process
4. Examples:
   - Classification (Iris Dataset)
   - Regression (Diabetes Dataset)

## 1. Loading and Preparing Data

First, let's import necessary modules and explore the DataLoader class capabilities.

In [1]:
import sys
sys.path.append("/Users/cube/Documents/Development/MAI/Сошников")
from framework.DataLoader import DataLoader

# Create DataLoader instance
dl = DataLoader()

# List available datasets
print("Available datasets:", dl.list_datasets())

Available datasets: ['iris', 'mnist', 'diabetes', 'wine']


### DataLoader Features:
- Loading different datasets
- Data preprocessing
- Batch creation

In [2]:
# Load Iris dataset
dl = DataLoader("iris")

# Get dataset statistics
stats = dl.get_stats()
print("\nDataset Statistics:")
for key, value in stats.items():
    print(f"{key}: {value}")

# View first few samples
data, labels = dl.head(3)
print("\nFirst 3 samples:")
print("Data:", data)
print("Labels:", labels)


Dataset Statistics:
data_shape: (150, 4)
labels_shape: (150,)
data_type: float64
labels_type: int64
unique_labels: [0 1 2]
data_mean: 3.4644999999999997
data_std: 1.9738430577598278

First 3 samples:
Data: [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]]
Labels: [0 0 0]


## 2. Neural Network Architecture

Our framework provides various components for building neural networks:

In [3]:
from framework.NeuralNetwork import NeuralNetwork, Layer, ActivationFunction, Optimizer

# Available Activation Functions:
print("Available Activation Functions:")
print("- ReLU")
print("- Sigmoid")
print("- Tanh")
print("- LeakyReLU")

# Available Optimizers:
print("\nAvailable Optimizers:")
print("- SGD")
print("- MomentumSGD")
print("- RMSprop")
print("- Adam")

Available Activation Functions:
- ReLU
- Sigmoid
- Tanh
- LeakyReLU

Available Optimizers:
- SGD
- MomentumSGD
- RMSprop
- Adam


### Creating a Neural Network

Let's create a simple neural network for classification:

In [4]:
# Define network parameters
input_size = 4  # Iris dataset has 4 features
hidden_size = 10
output_size = 3  # Iris has 3 classes

# Create optimizer
optimizer = Optimizer.Adam(learning_rate=0.01)

# Create neural network
model = NeuralNetwork(optimizer)

# Add layers
model.add_layer(Layer(input_size, hidden_size, ActivationFunction.ReLU))
model.add_layer(Layer(hidden_size, output_size, ActivationFunction.Sigmoid))

## 3. Training Process

Here's how to train your neural network:

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Prepare data
dl.shuffle()
X_train, X_test, y_train, y_test = train_test_split(dl.data, dl.labels, test_size=0.2)

# Normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Training parameters
epochs = 50
batch_size = 32

# Training loop
for epoch in range(epochs):
    epoch_loss = 0
    batches = dl.batch(batch_size)
    
    for X_batch, y_batch in batches:
        # Forward pass
        y_pred = model.forward(X_batch)
        
        # Convert labels to one-hot encoding
        y_one_hot = np.zeros((y_batch.size, output_size))
        y_one_hot[np.arange(y_batch.size), y_batch.astype(int)] = 1
        
        # Calculate loss
        loss = -np.sum(y_one_hot * np.log(y_pred + 1e-10)) / y_batch.size
        
        # Backward pass
        grad = (y_pred - y_one_hot) / y_batch.size
        model.backward(grad)
        
        # Update weights
        model.update()
        
        epoch_loss += loss
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss/len(batches):.4f}")

Epoch 10/50, Loss: 1.0384
Epoch 20/50, Loss: 0.6267
Epoch 30/50, Loss: 0.5410
Epoch 40/50, Loss: 0.4422
Epoch 50/50, Loss: 0.3420


## 4. Advanced Usage Tips

### Choosing Activation Functions
- ReLU: Good default choice for hidden layers
- Sigmoid: Use for binary classification output
- Tanh: Alternative to ReLU, works well with normalized data
- LeakyReLU: Helps prevent "dying ReLU" problem

### Optimizer Selection
- SGD: Simple but may converge slowly
- MomentumSGD: Better convergence than SGD
- RMSprop: Good for non-stationary objectives
- Adam: Best general-purpose optimizer

### Architecture Tips
1. Layer Sizing:
   - Input layer: Match feature dimension
   - Hidden layers: Usually powers of 2
   - Output layer: Match number of classes (classification) or 1 (regression)

2. Learning Rate:
   - Start with 0.01 for Adam
   - Start with 0.1 for SGD
   - Decrease if training is unstable